<h1 align=center><font size = 4>HW2: SVM,Naive Bayes and Decision Trees for Sentiment Classification</font></h1> 
<h2 align=center><font size = 3>【Machine Learning Course】</font></h2> 


- This is an individual assignment. However, you are allowed to discuss the problems with other students in the class. But you should write your own code and report.
- If you have any discussion with others, you should acknowledge the discussion in your report by mentioning their name.
- All the hyper-parameter details and other reporting details must be in the pdf report file only. <font color=red>You will have three types of files, 1) a single report (pdf), 2) dataset files (.txt), 3) a single code file (.ipynb). You should submit two files, one pdf file with report (<your-student-ID>.pdf) and one zip file with the dataset after preprocessing and codes (<your-student-ID>.zip)</font>. For the dataset format, please refer to the end of this notebook.
- You are free to use libraries with general utilities, such as matplotlib, numpy and scipy for python. You can use pre-existing implementation of the algorithms available in scikit-learn package. Do not use NLTK or any other NLP libraries for pre-processing.
## Sentiment Classification
In this assignment, we will design a sentiment classifier for classifying the sentiment of the reviews. This is a Natural Language Processing (NLP) task where the input is a natural language text and output is the sentiment label. We will consider the dataset yelp reviews for restaurants.
## Yelp dataset
The Yelp dataset consists of 7000 reviews in the training set, 1000 reviews in the validation set, and 2000 reviews in the test set. This is a 5 class problem where each review is classied into one of the five ratings with rating-5 being the best score and rating-1 being the worst score.




<h1 align=center><font size = 4>实践学习部分</font></h1> 

### 步骤1 引入必要的库

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import shuffle
from collections import defaultdict
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
import warnings 
warnings.filterwarnings("ignore") 

### 步骤2 数据预处理
Most of the algorithms described in the class expects input as a vector. However, the
reviews are natural language text of varying number of words. So the first step would
be to convert this varying length movie review to a fixed length vector representation.
We will consider two different ways of vectorizing the natural language text: binary
bag-of-words representation and frequency bag-of-words representation (as explained in
the end of the assignment). Convert both the datasets into both these representations.
Instruction for dataset submission is given in the end of the assignment .

In [14]:
# 
# This is to find the frequency map
# 
train_path = './yelp-train.txt'
with open(train_path, 'r') as f:
    lines = f.readlines()
    
dic = defaultdict(list)
for line in lines:
    sentence = line.split('\t')[0]
    sentiment = line.split('\t')[1][0]
    dic['sentence'].append(sentence)
    dic['sentiment'].append(sentiment)
                            
df = pd.DataFrame(dic)
# this is to punctuate the words
sets = set()
for s in df['sentence']:
    for word in s.split():
        sets.add(word.lower())
        
ls = list(sets)
punctuation = ['.',',',':',')','(','"',"'",'/','!','~','+','-','?','*']
for punc in punctuation:
    ls = list(sets)
    sets = set()
    for word in ls:
        for w in word.split(punc):
            if w:
                sets.add(w)
# this is to lowercase them and remove all not-word word
ls = list(sets)
sets = set()
for word in ls:
    flag = True
    word = word.lower()
    for w in word:
        if w < 'a' or w > 'z':
            flag = False
            break
    if flag:
        sets.add(word)
        
# calculate the frequency
newdic = defaultdict(int)
for s in df['sentence']:
    for word in s.split():
        if word in sets:
            newdic[word] += 1
newdic = dict(newdic)
so = list(newdic.items())
so.sort(key = lambda x:x[1], reverse=True)


# print the dictionary
so_list = list(so)
so_list = [list(item) for item in so_list]
so_list = list(enumerate(so_list,1))
#so_list = [list(item) for item in so_list]
output = list()

for i in range(0,len(so_list)):
    ite = list()
    ite.append(so_list[i][1][0])
    ite.append(so_list[i][0])
    ite.append(so_list[i][1][1])
    output.append(ite)



In [20]:
#生成词表 以下直至步骤三不建议重新运行，很慢，且不运行不影响后续分析
data=open("./yelp-vocab1.txt",'w+') 
for i in range(0,10000):
    print(output[i],file=data)
data.close()

path = './yelp-vocab1.txt'
with open(path, 'r') as f:
    lines = f.readlines()

data=open("./yelp-vocab.txt",'w+')
for line in lines:
    line=line.replace(",","\t").replace("[","").replace("]","").replace("'","")#改变元素
    print(line, file = data)
data.close()
#os.remove(r'./yelp-vocab1.txt') 这句话加在所有预处理完事儿

In [26]:
#生成train的替换后的文档
train_path = './yelp-train1.txt'
with open(train_path, 'r') as f:
    lines = f.readlines()
dic = defaultdict(list)
for line in lines:
    sentence = line.split('\t')[0]
    sentiment = line.split('\t')[1][0]
    dic['sentence'].append(sentence)
    dic['sentiment'].append(sentiment)
                            
df = pd.DataFrame(dic)
#os.remove(r'./yelp-train.txt')
for j in range(0,7000):
    words = list()
    for word in df['sentence'][j].split():
        words.append(word.lower())
    for word in range(0,len(words)):
        for i in range(0,10000):
            if words[word].lower() == so_list[i][1][0]:
                words[word] = str(so_list[i][0])

    data=open("./yelp-train.txt",'a')
    for word in words:
        word=word.replace(","," ").replace("[","").replace("]","").replace('"',"")#改变元素
        print(word, file = data, end = " ")
    print('\t',df['sentiment'][j], file = data)
    data.close()

In [31]:
#生成valid的替换后的文档
valid_path = './yelp-valid1.txt'
with open(valid_path, 'r') as f:
    lines = f.readlines()
dic = defaultdict(list)
for line in lines:
    sentence = line.split('\t')[0]
    sentiment = line.split('\t')[1][0]
    dic['sentence'].append(sentence)
    dic['sentiment'].append(sentiment)
#os.remove(r'./yelp-vaild.txt')                            
df = pd.DataFrame(dic)
for j in range(0,1000):
    words = list()
    for word in df['sentence'][j].split():
        words.append(word.lower())
    for word in range(0,len(words)):
        for i in range(0,10000):
            if words[word].lower() == so_list[i][1][0]:
                words[word] = str(so_list[i][0])

    data=open("./yelp-valid.txt",'a')
    for word in words:
        word=word.replace(","," ").replace("[","").replace("]","").replace('"',"")#改变元素
        print(word, file = data, end = " ")
    print('\t',df['sentiment'][j], file = data)
    data.close()

In [32]:
#生成test的替换后的文档
test_path = './yelp-test1.txt'
with open(test_path, 'r') as f:
    lines = f.readlines()
dic = defaultdict(list)
for line in lines:
    sentence = line.split('\t')[0]
    sentiment = line.split('\t')[1][0]
    dic['sentence'].append(sentence)
    dic['sentiment'].append(sentiment)
#os.remove(r'./yelp-vaild.txt')                            
df = pd.DataFrame(dic)
for j in range(0,2000):
    words = list()
    for word in df['sentence'][j].split():
        words.append(word.lower())
    for word in range(0,len(words)):
        for i in range(0,10000):
            if words[word].lower() == so_list[i][1][0]:
                words[word] = str(so_list[i][0])

    data=open("./yelp-test.txt",'a')
    for word in words:
        word=word.replace(","," ").replace("[","").replace("]","").replace('"',"")#改变元素
        print(word, file = data, end = " ")
    print('\t',df['sentiment'][j], file = data)
    data.close()

### 步骤3 yelp数据集：用BBOW构建模型
For this question, we will focus on the yelp dataset with binary bag-of-words (BBoW)
representation. We will use the F1-measure as the evaluation metric for the entire
assignment.

(a) As a baseline, report the performance of the random classifier (a classifier which
classifies a review into an uniformly random class) and the majority-class classifier
(a classifier which computes the majority class in the training set and classifies all
test instances as that majority class).


In [2]:
# 请在这里填写您的代码
# BBOW数据预处理
def bbowpreprocessing(dic):
    file_handle=open(dic,mode='r')
    content0=file_handle.readlines()
    #属性
    words_list=[[0 for col in range(10000)] for row in range(len(content0))]
    #目标
    sentiment=[0 for row in range(len(content0))]

    x1=''
    for i in range(len(content0)):
        sentiment[i]=content0[i][-2]
        for j in range(len(content0[i])-1):
            if content0[i][j].isdigit():
                x1+=content0[i][j]
            else:
                if x1!='':
                    x1=int(x1)
                    if 0<=x1<10000:
                        words_list[i][x1-1]=1
                    x1=''
    return words_list, sentiment


words_list_train, sentiment_train = bbowpreprocessing('./yelp-train.txt')
words_list_valid, sentiment_valid = bbowpreprocessing('./yelp-valid.txt')
words_list_test, sentiment_test = bbowpreprocessing('./yelp-test.txt')

In [10]:
# 随机分类器
randomclass_test = list()
for i in range(2000):
    randomclass_test.append(int(np.random.rand()*5)+1)
    
sentiment_test = [int(i) for i in sentiment_test]
f1 = f1_score(sentiment_test, randomclass_test, average = 'weighted')

print('随机分类器test的f1=',f1)

randomclass_train = list()
for i in range(7000):
    randomclass_train.append(int(np.random.rand()*5)+1)
    
sentiment_train = [int(i) for i in sentiment_train]
f1 = f1_score(sentiment_train, randomclass_train, average = 'weighted')

print('随机分类器train的f1=',f1)

randomclass_valid = list()
for i in range(1000):
    randomclass_valid.append(int(np.random.rand()*5)+1)
    
sentiment_valid = [int(i) for i in sentiment_valid]
f1 = f1_score(sentiment_valid, randomclass_valid, average = 'weighted')

print('随机分类器train的f1=',f1)

随机分类器test的f1= 0.20739646641749476
随机分类器train的f1= 0.21261962746532156
随机分类器train的f1= 0.1903313901283753


In [11]:
# 大类分类器
result = pd.value_counts(sentiment_train)
majorityclass = list()
for i in range(2000):
    majorityclass.append(result.index[0])

majorityclass = [int(i) for i in majorityclass]
f1 = f1_score(sentiment_test, majorityclass, average = 'weighted')
print('大类分类器test的f1=',f1)

result = pd.value_counts(sentiment_train)
majorityclass = list()
for i in range(7000):
    majorityclass.append(result.index[0])

majorityclass = [int(i) for i in majorityclass]
f1 = f1_score(sentiment_train, majorityclass, average = 'weighted')
print('大类分类器train的f1=',f1)

result = pd.value_counts(sentiment_train)
majorityclass = list()
for i in range(1000):
    majorityclass.append(result.index[0])

majorityclass = [int(i) for i in majorityclass]
f1 = f1_score(sentiment_valid, majorityclass, average = 'weighted')
print('大类分类器valid的f1=',f1)

大类分类器test的f1= 0.18238490007401925
大类分类器train的f1= 0.18380783390669325
大类分类器valid的f1= 0.18692625368731566


# (b) Now train Naive Bayes, Decision Trees, and Linear SVM for this task. 
[Note: You
should do a thorough hyper-parameter tuning by using the given validation set.
Also, note that you should use the appropriate naive Bayes classifier for binary
input features (also called Bernoulli naive Bayes).]

In [20]:
# 请在这里填写您的代码 (Naive Bayes)
Bern = BernoulliNB()
Bern.fit(words_list_train, sentiment_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [21]:
test_outcome_bern = Bern.predict(words_list_test)
test_outcome_bern = [int(i) for i in test_outcome_bern]
f1 = f1_score(sentiment_test, test_outcome_bern, average = 'weighted')
print('伯努利朴素贝叶斯test的f1=',f1)
test_outcome_bern = Bern.predict(words_list_train)
test_outcome_bern = [int(i) for i in test_outcome_bern]
f1 = f1_score(sentiment_train, test_outcome_bern, average = 'weighted')
print('伯努利朴素贝叶斯train的f1=',f1)
test_outcome_bern = Bern.predict(words_list_valid)
test_outcome_bern = [int(i) for i in test_outcome_bern]
f1 = f1_score(sentiment_valid, test_outcome_bern, average = 'weighted')
print('伯努利朴素贝叶斯valid的f1=',f1)

伯努利朴素贝叶斯test的f1= 0.3914862429730039
伯努利朴素贝叶斯train的f1= 0.5840324991638591
伯努利朴素贝叶斯valid的f1= 0.3643033176913067


In [4]:
# 请在这里填写您的代码 ( Decision Trees)
# 寻找最优参数
depth = list()
for i in range(5,13):
    depth.append(i)
DTC = GridSearchCV(DecisionTreeClassifier(), param_grid={"class_weight": ["balanced"], "criterion": ("gini", 'entropy'),\
      "splitter": ['random', 'best'], "max_depth": depth, 'random_state': [1]})
DTC.fit(words_list_train, sentiment_train)
validpred = DTC.predict(words_list_valid)
#sigma = metrics.mean_squared_error(sentiment_valid, validpred) #计算均方差
#print(sigma)
print('the best patams is{}'.format(DTC.best_params_))

the best patams is{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 12, 'random_state': 1, 'splitter': 'best'}


In [17]:
#最优参数代入
DTC = DecisionTreeClassifier(criterion='gini', max_depth=12, class_weight = 'balanced', splitter = 'best', random_state = 1)
DTC.fit(words_list_train, sentiment_train)
# test_outcome_bern = [int(i) for i in test_outcome_bern]

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [19]:
DTC = DecisionTreeClassifier(criterion='gini', max_depth=12, class_weight = 'balanced', splitter = 'best', random_state = 1)
DTC.fit(words_list_train, sentiment_train)
test_outcome_bern_train = DTC.predict(words_list_train)
test_outcome_bern_train = [int(i) for i in test_outcome_bern_train]
f1 = f1_score(sentiment_train, test_outcome_bern_train, average = 'weighted')
print('决策树train的f1=',f1)
test_outcome_bern_valid = DTC.predict(words_list_valid)
test_outcome_bern_valid = [int(i) for i in test_outcome_bern_valid]
f1 = f1_score(sentiment_valid, test_outcome_bern_valid, average = 'weighted')
print('决策树valid的f1=',f1)

决策树test的f1= 0.31142727259250935
决策树train的f1= 0.47581208772663364
决策树valid的f1= 0.3023970557648267


In [14]:
# 请在这里填写您的代码 ( Linear SVM)
# 寻找最优参数
svr = GridSearchCV(LinearSVC(), param_grid={"C": np.logspace(-3, 3, 7)})

svr.fit(words_list_train, sentiment_train) #训练
sentiment_pred = svr.predict(words_list_valid)   #测试集测试

In [14]:
print('the best parameter is{}'.format(svr.best_params_))

the best parameter is{'C': 0.01}


In [22]:
#最优参数代入
LSVC = LinearSVC(C = 0.01)
LSVC.fit(words_list_train, sentiment_train)

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [23]:
test_outcome_LSVC = LSVC.predict(words_list_test)
#test_outcome_LSVC = [int(i) for i in test_outcome_LSVC]
f1 = f1_score(sentiment_test, test_outcome_LSVC, average = 'weighted')
print('线性支持向量机test的f1=',f1)
test_outcome_LSVC = LSVC.predict(words_list_train)
#test_outcome_LSVC = [int(i) for i in test_outcome_LSVC]
f1 = f1_score(sentiment_train, test_outcome_LSVC, average = 'weighted')
print('线性支持向量机train的f1=',f1)
test_outcome_LSVC = LSVC.predict(words_list_valid)
#test_outcome_LSVC = [int(i) for i in test_outcome_LSVC]
f1 = f1_score(sentiment_valid, test_outcome_LSVC, average = 'weighted')
print('线性支持向量机valid的f1=',f1)

线性支持向量机test的f1= 0.4710937321447393
线性支持向量机train的f1= 0.8065874229852863
线性支持向量机valid的f1= 0.45023726619464627


(c) Report the list of hyper-parameters you considered for each classifier, the range of the individual hyper-parameters and the best value for these hyper-parameters chosen based on the validation set performance1.

(d) Report training, validation, and test F1-measure for all the classifiers (with best hyper-parameter configuration).

(e) Comment about the performance of different classifiers. Why did a particular
classifier performed better than the others? What was the role of that hyper-
parameter that fetched you the best results.

### 步骤4 yelp数据集：用FBoW构建模型
Now we will repeat step 3 but with frequency bag-of-words (FBoW) representation.

(a) Train Naive Bayes, Decision Trees, and Linear SVM for this task. [Note: You
should do a thorough hyper-parameter tuning by using the given validation set. Also, note that you should use the appropriate naive Bayes classifier for real valued
input features (also called Gaussian naive Bayes).]

In [6]:
# 请在这里填写您的代码
# FBOW数据预处理
def fbowpreprocessing(dic):
    file_handle=open(dic,mode='r')
    content0=file_handle.readlines()
    #属性
    words_list=[[0 for col in range(10000)] for row in range(len(content0))]
    #目标
    sentiment=[0 for row in range(len(content0))]

    x1=''
    for i in range(len(content0)):
        sentiment[i]=content0[i][-2]
        for j in range(len(content0[i])-1):
            if content0[i][j].isdigit():
                x1+=content0[i][j]
            else:
                if x1!='':
                    x1=int(x1)
                    if 0<=x1<10000:
                        words_list[i][x1-1]+=1
                    x1=''
    sum = 0
    for i in range(10000):
        sum += words_list[0][i]
    for i in range(10000):
        words_list[0][i] = words_list[0][i]/sum
    return words_list, sentiment

words_list_train_fbow, sentiment_train_fbow = fbowpreprocessing('./yelp-train.txt')
words_list_valid_fbow, sentiment_valid_fbow = fbowpreprocessing('./yelp-valid.txt')
words_list_test_fbow, sentiment_test_fbow = fbowpreprocessing('./yelp-test.txt')

In [24]:
# 请在这里填写您的代码 ( Naive Bayes)
Gau = GaussianNB()
Gau.fit(words_list_train_fbow, sentiment_train_fbow)

GaussianNB(priors=None, var_smoothing=1e-09)

In [25]:
test_outcome_gau = Gau.predict(words_list_test_fbow)
#test_outcome_gau = [int(i) for i in test_outcome_gau]
f1 = f1_score(sentiment_test_fbow, test_outcome_gau, average = 'weighted')
print('高斯朴素贝叶斯test的f1=',f1)
test_outcome_gau = Gau.predict(words_list_train_fbow)
#test_outcome_gau = [int(i) for i in test_outcome_gau]
f1 = f1_score(sentiment_train_fbow, test_outcome_gau, average = 'weighted')
print('高斯朴素贝叶斯train的f1=',f1)
test_outcome_gau = Gau.predict(words_list_valid_fbow)
#test_outcome_gau = [int(i) for i in test_outcome_gau]
f1 = f1_score(sentiment_valid_fbow, test_outcome_gau, average = 'weighted')
print('高斯朴素贝叶斯valid的f1=',f1)

高斯朴素贝叶斯test的f1= 0.2760047772674479
高斯朴素贝叶斯train的f1= 0.6651035134883894
高斯朴素贝叶斯valid的f1= 0.25636765458952404


In [7]:
# 请在这里填写您的代码 ( Decision Trees)
# 寻找最优参数
depth = list()
for i in range(5,13):
    depth.append(i)
DTC = GridSearchCV(DecisionTreeClassifier(), param_grid={"class_weight": ["balanced"], "criterion": ("gini", 'entropy'),\
      "splitter": ['random','best'], "max_depth": depth, 'random_state': [1]})
DTC.fit(words_list_train_fbow, sentiment_train_fbow)
validpred = DTC.predict(words_list_valid_fbow)
#sigma = metrics.mean_squared_error(sentiment_valid_fbow, validpred) #计算均方差
#print(sigma)
print('the best parameters are{}'.format(DTC.best_params_))

the best parameters are{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 7, 'random_state': 1, 'splitter': 'random'}


In [26]:
#最优参数代入
DTC = DecisionTreeClassifier(criterion='gini', max_depth=7, class_weight = 'balanced', splitter = 'random', random_state = 1)
DTC.fit(words_list_train_fbow, sentiment_train_fbow)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=7, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='random')

In [28]:
test_outcome = DTC.predict(words_list_test_fbow)
# test_outcome = [int(i) for i in test_outcome]
f1 = f1_score(sentiment_test_fbow, test_outcome, average = 'weighted')
print('决策树test的f1=',f1)
test_outcome = DTC.predict(words_list_train_fbow)
# test_outcome = [int(i) for i in test_outcome]
f1 = f1_score(sentiment_train_fbow, test_outcome, average = 'weighted')
print('决策树train的f1=',f1)
test_outcome = DTC.predict(words_list_valid_fbow)
# test_outcome = [int(i) for i in test_outcome]
f1 = f1_score(sentiment_valid_fbow, test_outcome, average = 'weighted')
print('决策树valid的f1=',f1)

决策树test的f1= 0.33034472621680333
决策树train的f1= 0.37573213516397896
决策树valid的f1= 0.3262174480754342


In [36]:
# 请在这里填写您的代码 ( Linear SVM)
# 寻找最优参数
svr = GridSearchCV(LinearSVC(), param_grid={"C": np.logspace(-3, 3, 7)})

svr.fit(words_list_train_fbow, sentiment_train_fbow) #训练
sentiment_pred = svr.predict(words_list_valid_fbow)   #测试集测试
print('the best parameter is{}'.format(svr.best_params_))


the best parameter is{'C': 0.01}


In [29]:
#最优参数代入
LSVC = LinearSVC(C = 0.01)
LSVC.fit(words_list_train_fbow, sentiment_train_fbow)

LinearSVC(C=0.01, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [30]:
test_outcome_LSVC = LSVC.predict(words_list_test_fbow)
#test_outcome_LSVC = [int(i) for i in test_outcome_LSVC]
f1 = f1_score(sentiment_test_fbow, test_outcome_LSVC, average = 'weighted')
print('线性支持向量机test的f1=',f1)
test_outcome_LSVC = LSVC.predict(words_list_train_fbow)
#test_outcome_LSVC = [int(i) for i in test_outcome_LSVC]
f1 = f1_score(sentiment_train_fbow, test_outcome_LSVC, average = 'weighted')
print('线性支持向量机train的f1=',f1)
test_outcome_LSVC = LSVC.predict(words_list_valid_fbow)
#test_outcome_LSVC = [int(i) for i in test_outcome_LSVC]
f1 = f1_score(sentiment_valid_fbow, test_outcome_LSVC, average = 'weighted')
print('线性支持向量机valid的f1=',f1)

线性支持向量机test的f1= 0.46979351639773886
线性支持向量机train的f1= 0.8131439860094576
线性支持向量机valid的f1= 0.4623409504195169


(b) Report the list of hyper-parameters you considered for each classifier, the range of the individual hyper-parameters and the best value for these hyper-parameters
chosen based on the validation set performance.

(c) Report training, validation, and test F1-measure for all the classifiers (with best hyper-parameter configuration).

(d) Comment about the performance of different classifiers. Why did a particular
classifier performed better than the others? What was the role of that hyper-
parameter that fetched you the best results.

(e) Compare the performance with the binary bag-of-words based classifiers. Why the
difference in performance? Give a brief explanation comparing BBoW Naive Bayes
and FBoW Naive Bayes and similarly for Decision Trees and Linear SVM.

(f) Which representation is better? Why?


<h1 align=center><font size = 4>实验补充说明</font></h1> 

## Instruction for binary bag-of-words representation
1. First step is to construct the word vocabulary for a dataset. Given a dataset, we
will only consider the training set and enumerate all unique words in the training
set reviews. To do this, we should do a bit of pre-processing to the raw text. For
this assignment, we will do only 2 pre-processing steps: removal of punctuation, and
lower-casing the words. For example, (How old are you?) would become (how old are
you).
2. Once you have the vocabulary, count the frequency of each word in the training set
and sort the words in the vocabulary based on frequency (in descending order). Now
pick top 10,000 words in the vocabulary and ignore the rest of the words. These 10,000
words will form the feature set for our classification process.
3. For any example to be classified, generate a 10,000 dimensional feature vector as follows: for each of the top 10,000 words, there is one corresponding dimension in the
feature vector that is 1 if the example contains the word, and 0 otherwise. Make sure
to use the same pre-processing steps as before for validation and test reviews.

## Instruction for frequency bag-of-words representation
1. Follow steps 1-2 of the instructions for the binary bag-of-words.
2. Now, for each of the 10,000 words, the corresponding feature is the frequency of occurrence of that word in the given review. You can calculate this by summing the
occurrences of words in a review into a histogram, and than divide by the sum of
occurrences of all 10,000 words so that the vector for each example sums to 1.

## Instruction for dataset submission
You should submit yelp-train.txt, yelp-valid.txt, yelp-test.txt,yelp-vocab.txt.
1. yelp-vocab.txt file should contain the 10,000 words in the vocabulary, their corresponding id (starting from 1), and their frequency. Each line is a word, its numeric id,
and its frequency all tab separated. Example:  
the 1 20456  
where the is the word, 1 is the id of the word, and 20456 is the frequency of the
word.
2. For train/valid/test file, each line is a data point. Each review should be represented
as space separated ids for corresponding words in the review and the class label in
mentioned in the end of the review as tab separated. Skip ids for words which are not
there in top 10000. Just replace the word with the id (don't sort the ids in a row).
Example:  
100 8 3 1034 0   
Here 0 is the class label and rest of the numbers represent a 4 word review.

## Instruction for code submission
1. Submit a single zipped folder with your student id as the name of the folder. For
example if your student ID is 12345678, then the submission should be 12345678.zip.
2. You can only use python. You should work with our provided .ipynb file and  submit your solution as a jupyter notebook.
3. <font color=red>Make sure all the data files needed to run your code is within the folder and loaded with
relative path. We should be able to run your code without making any modifications</font>.

## Instruction for report submission
1. You report should be brief and to the point. When asked for comments, your comment
should not be more than 3-4 lines.
2. Do not include your code in the report!